<a href="https://colab.research.google.com/github/mimilazarova/DD2424-covid-xray-project/blob/master/playground_colab_mimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import os


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:

with open('/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/data/trainX1.pickle', 'rb') as f:
  X1 = pickle.load(f)
with open('/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/data/trainX2.pickle', 'rb') as f:
  X2 = pickle.load(f)

trainX = np.concatenate((X1, X2))

with open('/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/data/testX.pickle', 'rb') as f:
  testX = pickle.load(f)
with open('/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/data/train_y.pickle', 'rb') as f:
  train_y = pickle.load(f)
with open('/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/data/test_y.pickle', 'rb') as f:
  test_y = pickle.load(f)

N = trainX.shape[0]
testN = testX.shape[0]

trainX = trainX.reshape((N, 224, 224, 1))
testX = testX.reshape((testN, 224, 224, 1))

# trainX, valX, train_y, val_y = train_test_split(trainX, train_y, test_size=0.2, random_state=42, shuffle=True, stratify=None)

# print(sum(train_y==0)/len(train_y))
# print(sum(train_y==1)/len(train_y))
# print(sum(train_y==2)/len(train_y))

# print(sum(val_y==0)/len(val_y))
# print(sum(val_y==1)/len(val_y))
# print(sum(val_y==2)/len(val_y))

# args = [train_y==0]
# X0, vX0, y0, vy0 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

# args = [train_y==1]
# X1, vX1, y1, vy1 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

# args = [train_y==2]
# X2, vX2, y2, vy2 = train_test_split(trainX[args], train_y[args],  test_size=0.1, random_state=42) 

# trainX = np.concatenate((X0, X1, X2))
# train_y = np.concatenate((y0, y1, y2))

# valX = np.concatenate((vX0, vX1, vX2))
# val_y = np.concatenate((vy0, vy1, vy2))

trainX.shape

In [0]:
ids = np.argwhere(train_y==1)
choices = np.random.choice(ids[:, 0], 10*np.sum(train_y==2), replace=False)

X1 = trainX[choices]
y1 = train_y[choices]

ids = np.argwhere(train_y==0)
choices = np.random.choice(ids[:, 0], 10*np.sum(train_y==2), replace=False)

X0 = trainX[choices]
y0 = train_y[choices]

X2 = trainX[train_y==2]
y2 = train_y[train_y==2]

trainX_balanced = np.concatenate((X0, X1, X2))
train_y_balanced = np.concatenate((y0, y1, y2))
trainX_balanced.shape

trainX, valX, train_y, val_y = train_test_split(trainX_balanced, train_y_balanced, test_size=0.2, random_state=42, shuffle=True, stratify=None)

In [0]:
bs = 16
init_lr = 2e-4
nepoch = 50
N = len(train_y)

optimizer = keras.optimizers.Adam(init_lr)

class_weights = {}
# for c in range(3):
#   class_weights[c] = 1000/np.sum(train_y==c)
#   print(class_weights[c])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

aug = keras.preprocessing.image.ImageDataGenerator(rotation_range=20, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.01, horizontal_flip=True,
                         vertical_flip=False, fill_mode="nearest")


In [0]:
# one-hot-encoded vectors, not sure it we need them, the fitting the models with them did not work for me

# trainY = np.zeros((N, 3))
# testY = np.zeros((testN, 3))

# trainY[np.arange(N), train_y] = 1
# testY[np.arange(testN), test_y] = 1


In [0]:
model2 = keras.models.Sequential()
model2.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu', input_shape=(224, 224, 1)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(3))

class_weights[0] = 1
class_weights[1] = 2
class_weights[2] = 10

model2.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])




# history2 = model2.fit(trainX, train_y, batch_size = bs, epochs= nepoch,  callbacks=[early_stopping],
#                     class_weight=class_weights, validation_data=(valX, val_y))

# history2 = model2.fit(trainX_balanced, train_y_balanced, 
#                       batch_size = bs, epochs= nepoch, 
#                       callbacks=[early_stopping],
#                       validation_data=(valX, val_y))

history2 = model2.fit(aug.flow(trainX, train_y, batch_size=bs), 
                      epochs=nepoch, callbacks=[early_stopping],
                      class_weight=class_weights,
                      steps_per_epoch = int(N/bs),
                      validation_data=(valX, val_y))

In [0]:
plt.plot(history2.history['accuracy'], label='accuracy')
plt.plot(history2.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_y_pred_m = model2.predict(testX)
test_y_pred2 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred2, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred2, digits=3))

In [0]:
model3 = keras.models.Sequential()
model3.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu', input_shape=(224, 224, 1)))
model3.add(layers.MaxPooling2D((4, 4)))
model3.add(layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model3.add(layers.MaxPooling2D((4, 4)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.Flatten())
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(32, activation='relu'))
model3.add(layers.Dense(3))

class_weights[0] = 1
class_weights[1] = 1
class_weights[2] = 4



model3.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# history3 = model3.fit(trainX, train_y, 
#                       batch_size = bs, epochs=nepoch,
#                       callbacks=[early_stopping],
#                       class_weight=class_weights,
#                       validation_data=(valX, val_y))

# history3 = model3.fit(trainX_balanced, train_y_balanced, 
#                       batch_size = bs, epochs=nepoch, 
#                       callbacks=[early_stopping],
#                       validation_data=(valX, val_y))

history3 = model3.fit(aug.flow(trainX, train_y, batch_size=bs), 
                      epochs=nepoch, callbacks=[early_stopping],
                      class_weight=class_weights,
                      steps_per_epoch = int(N/bs),
                      validation_data=(valX, val_y))  

plt.plot(history3.history['accuracy'], label='accuracy')
plt.plot(history3.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_y_pred_m = model3.predict(testX)
test_y_pred3 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred3, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred3, digits=3))

In [0]:
# X_input = layers.Input((224, 224, 1))

# X = layers.Conv2D(64, (7, 7), strides=(2, 2), padding = 'same', activation='elu', kernel_initializer='he_normal')(X_input)

# X1 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation='elu', kernel_initializer='he_normal')(X)

# pepx1 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(X1)
# pepx1 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx1)
# pepx1 = layers.DepthwiseConv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx1)
# pepx1 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx1)
# pepx1 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx1)

# X = layers.Add()([pepx1, X1])

# X2 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation='elu', kernel_initializer='he_normal')(X)

# pepx2 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(X2)
# pepx2 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx2)
# pepx2 = layers.DepthwiseConv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx2)
# pepx2 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx2)
# pepx2 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx2)


# X = layers.Add()([pepx2, X2])
# X3 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', activation='elu', kernel_initializer='he_normal')(X)

# pepx3 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(X3)
# pepx3 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx3)
# pepx3 = layers.DepthwiseConv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx3)
# pepx3 = layers.Conv2D(16, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx3)
# pepx3 = layers.Conv2D(256, (1, 1), padding='same', activation='elu', kernel_initializer='he_normal')(pepx3)

# X = layers.Flatten()(pepx3)
# X = layers.Dense(128, activation='elu', kernel_initializer='he_normal')(X)
# X = layers.Dense(32, activation='elu', kernel_initializer='he_normal')(X)
# X = layers.Dense(3, activation='softmax', kernel_initializer='he_normal')(X)

# model5 = keras.models.Model(inputs=X_input, outputs=X)
# model5.summary()

In [0]:
# for c in range(3):
#   class_weights[c] = np.sum(train_y==(2-c))/N
#   print(class_weights[c])


# model5.compile(optimizer=optimizer,
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])


# # history5 = model5.fit(trainX, train_y, batch_size = bs,
# #                       epochs=5, callbacks=[early_stopping],
# #                       class_weight=class_weights, validation_data=(valX, val_y))  

# history5 = model5.fit(aug.flow(trainX, train_y, batch_size = bs),
#                       epochs=5, callbacks=[early_stopping],
#                       steps_per_epoch = int(N/bs),
#                       class_weight=class_weights,
#                       validation_data=(valX, val_y))

# plt.plot(history5.history['accuracy'], label='accuracy')
# plt.plot(history5.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

# test_y_pred_m = model5.predict(testX)
# test_y_pred5 = np.argmax(test_y_pred_m, axis=1)

# conf_matrix = confusion_matrix(test_y, test_y_pred5, labels=[0, 1, 2])
# print("Confusion matrix:")
# print(conf_matrix)
# print(classification_report(test_y, test_y_pred5, digits=3))


In [0]:
N = train_y.shape[0]
testN = test_y.shape[0]
for c in range(3):
  print(np.sum(test_y==c)/testN)
print("\n")
for c in range(3):
  print(np.sum(train_y==c)/N)

In [0]:
X_input = layers.Input((224, 224, 1))

X = layers.Conv2D(64, (7, 7), strides=(2, 2), padding = 'same', kernel_initializer='he_uniform')(X_input)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)

X1 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_uniform')(X)
X1 = layers.BatchNormalization()(X1)

b1 = layers.MaxPool2D((2,2))(X) 
b1 = layers.Conv2D(16, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)
b1 = layers.Activation('relu')(b1)
b1 = layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)


X = layers.Add()([X1, b1])
X = layers.Activation('relu')(X)

X2 = layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
X2 = layers.BatchNormalization()(X2)

b2 = layers.MaxPool2D((2,2))(X) 
b2 = layers.Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)
b2 = layers.Activation('relu')(b2)
b2 = layers.Conv2D(512, (1, 1), padding='same',  kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)


X = layers.Add()([X2, b2])
X = layers.Activation('relu')(X)


X = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.MaxPool2D(2, 2)(X)

X = layers.Flatten()(X)
X = layers.Dense(128, kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.Dense(3,kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('softmax')(X)

model6 = keras.models.Model(inputs=X_input, outputs=X, name='model_6')
model6.summary()

In [0]:
class_weights[0] = 1
class_weights[1] = 1
class_weights[2] = 4


model6.compile(optimizer= optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# history6 = model6.fit(trainX, train_y,
#                       batch_size=bs, epochs=nepoch,
#                       callbacks=[early_stopping],
#                       class_weight=class_weights,
#                       validation_data=(valX, val_y))  

history6 = model6.fit(aug.flow(trainX, train_y, batch_size = bs),
                      epochs=nepoch, callbacks=[early_stopping],
                      steps_per_epoch = int(N/bs),
                      class_weight=class_weights,
                      validation_data=(valX, val_y))


In [0]:
plt.plot(history6.history['accuracy'], label='accuracy')
plt.plot(history6.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_y_pred_m = model6.predict(testX)
test_y_pred6 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred6, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred6, digits=3))

In [0]:
X_input = layers.Input((224, 224, 1))

X = layers.Conv2D(64, (7, 7), strides=(2, 2), padding = 'same', kernel_initializer='he_uniform')(X_input)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)

X1 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_uniform')(X)
X1 = layers.BatchNormalization()(X1)

b1 = layers.MaxPool2D((2,2))(X) 
b1 = layers.Conv2D(16, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)
b1 = layers.Activation('relu')(b1)
b1 = layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)


X = layers.Add()([X1, b1])
X = layers.Activation('relu')(X)

X2 = layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
X2 = layers.BatchNormalization()(X2)

b2 = layers.MaxPool2D((2,2))(X) 
b2 = layers.Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)
b2 = layers.Activation('relu')(b2)
b2 = layers.Conv2D(512, (1, 1), padding='same',  kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)


X = layers.Add()([X2, b2])
X = layers.Activation('relu')(X)

X3 = layers.Conv2D(1024, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
X3 = layers.BatchNormalization()(X3)

b3 = layers.MaxPool2D((2,2))(X) 
b3 = layers.Conv2D(64, (1, 1), padding='same', kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)
b3 = layers.Activation('relu')(b3)
b3 = layers.Conv2D(1024, (1, 1), padding='same',  kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)


X = layers.Add()([X3, b3])
X = layers.Activation('relu')(X)

X = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.MaxPool2D(2, 2)(X)

X = layers.Flatten()(X)
X = layers.Dense(128, kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.Dense(3,kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('softmax')(X)

model7 = keras.models.Model(inputs=X_input, outputs=X, name='model_7')
model7.summary()

In [0]:
class_weights = {}
for c in range(3):
  class_weights[c] = 1000/np.sum(train_y==c)
  print(class_weights[c])
# class_weights[0] = 1
# class_weights[1] = 2
# class_weights[2] = 8

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


model7.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# history7 = model7.fit(trainX, train_y,
#                       batch_size=bs, epochs=nepoch,
#                       callbacks=[early_stopping],
#                       class_weight=class_weights,
#                       validation_data=(valX, val_y))  

history7 = model7.fit(aug.flow(trainX, train_y, batch_size = bs),
                      epochs=nepoch, callbacks=[early_stopping],
                      steps_per_epoch = int(N/bs),
                      class_weight=class_weights,
                      validation_data=(valX, val_y))


plt.plot(history7.history['accuracy'], label='accuracy')
plt.plot(history7.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_y_pred_m = model7.predict(testX)
test_y_pred7 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred7, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred7, digits=3))

In [0]:
X_input = layers.Input((224, 224, 1))

X = layers.Conv2D(64, (7, 7), strides=(2, 2), padding = 'same', kernel_initializer='he_uniform')(X_input)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)

X1 = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_uniform')(X)
X1 = layers.BatchNormalization()(X1)

b1 = layers.MaxPool2D((2,2))(X) 
b1 = layers.Conv2D(16, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)
b1 = layers.Activation('relu')(b1)
b1 = layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)

X = layers.Add()([X1, b1])
X = layers.Activation('relu')(X)

X1 = layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform')(X)
X1 = layers.BatchNormalization()(X1)

b1 = layers.Conv2D(16, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)
b1 = layers.Activation('relu')(b1)
b1 = layers.Conv2D(256, (1, 1), padding='same', kernel_initializer='he_uniform')(b1)
b1 = layers.BatchNormalization()(b1)


X = layers.Add()([X1, b1])
X = layers.Activation('relu')(X)

X2 = layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
X2 = layers.BatchNormalization()(X2)

b2 = layers.MaxPool2D((2,2))(X) 
b2 = layers.Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)
b2 = layers.Activation('relu')(b2)
b2 = layers.Conv2D(512, (1, 1), padding='same',  kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)


X = layers.Add()([X2, b2])
X = layers.Activation('relu')(X)

X2 = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal')(X)
X2 = layers.BatchNormalization()(X2)

b2 = layers.Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)
b2 = layers.Activation('relu')(b2)
b2 = layers.Conv2D(512, (1, 1), padding='same',  kernel_initializer='he_uniform')(b2)
b2 = layers.BatchNormalization()(b2)

X = layers.Add()([X2, b2])
X = layers.Activation('relu')(X)

X3 = layers.Conv2D(1024, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
X3 = layers.BatchNormalization()(X3)

b3 = layers.MaxPool2D((2,2))(X) 
b3 = layers.Conv2D(64, (1, 1), padding='same', kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)
b3 = layers.Activation('relu')(b3)
b3 = layers.Conv2D(1024, (1, 1), padding='same',  kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)


X = layers.Add()([X3, b3])
X = layers.Activation('relu')(X)

X3 = layers.Conv2D(1024, (3, 3), padding='same', kernel_initializer='he_normal')(X)
X3 = layers.BatchNormalization()(X3)

b3 = layers.Conv2D(64, (1, 1), padding='same', kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)
b3 = layers.Activation('relu')(b3)
b3 = layers.Conv2D(1024, (1, 1), padding='same',  kernel_initializer='he_uniform')(b3)
b3 = layers.BatchNormalization()(b3)

X = layers.Add()([X3, b3])
X = layers.Activation('relu')(X)

X = layers.Conv2D(512, (3, 3), padding='same', kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.MaxPool2D(2, 2)(X)

X = layers.Flatten()(X)
X = layers.Dense(128, kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('relu')(X)
X = layers.Dense(3,kernel_initializer='he_uniform')(X)
X = layers.BatchNormalization()(X)
X = layers.Activation('softmax')(X)

model8 = keras.models.Model(inputs=X_input, outputs=X, name='model_8')
#model8.summary()

In [0]:
class_weights[0] = 1
class_weights[1] = 1
class_weights[2] = 10

# metrics = [keras.metrics.TopKCategoricalAccuracy(), keras.metrics.Accuracy()]

dirname = "/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/"

filename = "model8-undersampling-weights-augmentation.hdf5"

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(dirname, filename),  
                                        monitor="val_accuracy",
                                        safe_best_only = True,
                                        verbose=1)

model8.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# history8 = model8.fit(trainX, train_y, batch_size=8,
#                       epochs=25, callbacks=[early_stopping],
#                       class_weight=class_weights, validation_data=(valX, val_y))
  
# history8 = model8.fit(trainX_balanced, train_y_balanced, 
#                       batch_size = 8, epochs=25, 
#                       callbacks=[early_stopping],
#                       validation_data=(valX, val_y))

history8 = model8.fit(aug.flow(trainX, train_y, batch_size = bs),
                      epochs=nepoch, callbacks=[early_stopping, checkpoint],
                      steps_per_epoch = int(N/bs),
                      class_weight=class_weights,
                      validation_data=(valX, val_y))



plt.plot(history8.history['accuracy'], label='accuracy')
plt.plot(history8.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


val_y_pred_m = model8.predict(valX)
val_y_pred8 = np.argmax(val_y_pred_m, axis=1)

conf_matrix = confusion_matrix(val_y, val_y_pred8, labels=[0, 1, 2])
print("Validation set")
print(conf_matrix)
print(classification_report(val_y, val_y_pred8, digits=3))

test_y_pred_m = model8.predict(testX)
test_y_pred8 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred8, labels=[0, 1, 2])
print("Test set")
print(conf_matrix)
print(classification_report(test_y, test_y_pred8, digits=3))

In [0]:
dirname = "/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/"
filename = "model8-3-augmentation-16-0.78.hdf5"
m = keras.models.load_model(os.path.join(dirname, filename))

val_y_pred_m = m.predict(valX)
val_y_pred8 = np.argmax(val_y_pred_m, axis=1)

conf_matrix = confusion_matrix(val_y, val_y_pred8, labels=[0, 1, 2])
print("Validation set")
print(conf_matrix)
print(classification_report(val_y, val_y_pred8, digits=3))

test_y_pred_m = m.predict(testX)
test_y_pred8 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred8, labels=[0, 1, 2])
print("Test set")
print(conf_matrix)
print(classification_report(test_y, test_y_pred8, digits=3))

In [0]:
# filename = "/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/model8.h5"
# model8.save(filename)

In [0]:
filename = "/content/drive/My Drive/KTH/Year 1/DD2424 Deep Learning/DD2424_Project/model8-16-0.91.hdf5"
loaded = keras.models.load_model(filename)
test_y_pred_m = loaded.predict(testX)
test_y_pred8 = np.argmax(test_y_pred_m, axis=1)

conf_matrix = confusion_matrix(test_y, test_y_pred8, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, test_y_pred8, digits=3))